# Convert hierarchical JSON to pd.DataFrame

**Task**: given a nested, hierarchical JSON (or Python `dict`), normalize it
to a usable table representation such as a valid `pandas.DataFrame`.

**Sample dict**:
```python
po_dict = {'ABC': {"open": {},
                   "updated":
                       {"12": {"22334": 5, "22335": 5},
                        "01": {"22335": 1, "22336": 2, "22337": 2},
                        "02": {"22337": 8, "order": 2},
                        "03": {"order": 5}}},
            'DEF': {"open": {},
                    "updated":
                       {"12": {"22338": 5, "22339": 5},
                        "01": {"22339": 1, "22340": 2, "22341": 2},
                        "02": {"22341": 8, "order": 2},
                        "03": {"order": 5}}},
                    }
```

**Expected outcome**:
```
material | state   | month | po_nb | po_value
---------------------------------------------
ABC      | open    |       |       |
ABC      | updated | 12    | 22334 | 5
ABC      | updated | 12    | 22335 | 5
ABC      | updated | 1     | 22335 | 1
ABC      | updated | 1     | 22336 | 2
ABC      | updated | 1     | 22337 | 2
ABC      | updated | 2     | 22337 | 8
ABC      | updated | 2     | order | 2
ABC      | updated | 3     | order | 5
DEF      | open    |       |       |
...
```

In [1]:
# sample sparse dict

sparse_sample_dict = {
  "ABC": {
    "open": {},
    "updated": {
      "12": {
        "22334": 5,
        "22335": 5
      },
      "01": {
        "22335": 1,
        "22336": 2,
        "22337": 2
      },
      "02": {
        "22337": 8,
        "order": 2
      },
      "03": {
        "order": 5
      }
    }
  },
  "DEF": {
    "open": {},
    "updated": {
      "12": {"22338": 5, "22339": 5},
      "01": {"22339": 1, "22340": 2, "22341": 2},
      "02": {"22341": 8, "order": 2},
      "03": {"order": 5}
    }
  }
}

In [2]:
# PoC for recursive "flattening" of a sparse dict into a list of
#  lists (LOL) where each inner list equals one row (ROW) in the
#  new table

def rec(d, row=None, lol= None):
    print(f"\nfunction called; got:\n d: {d};\n ROW: {row};\n LOL: {lol}")

    if not row:
        row = []
    if not lol:
        lol = []

    for k, v in d.items():

        row.append(k)
        print(f"\nadded key {k} to ROW:\n{row}")

        if v and isinstance(v, dict):
            # row continues, so...
            print(f"\nrow continues; recursing...")
            # 1. go deeper and receive updated row & lol
            row, lol = rec(d=v, row=row, lol=lol)
            print(f"\nreceived:\n ROW: {row}\n LOL: {lol}")

        else:
            # row ends, so...
            # 1. add current val to row
            row.append(v)
            print(f"\nROW ends here; added {v} to ROW:\n{row}")
            # 2. add finalized row to lol
            lol.append(row[:])
            print(f"\nappended finalized row to LOL:\n{lol}")
            # 3. remove latest cell from row
            row.pop()
            print(f"\nremoved latest entry from ROW again:\n{row}")

        # current value processed, so...
        print(f"\ncurrent val {v} processed.")
        # 1. remove latest level from row
        row.pop()
        print(f"\nremoved latest level from ROW:\n{row}")

    # all items processed, so...
    print(f"\nall items iterated; returning:")
    # 1. return updated row & lol
    print(f" ROW:\n{row}")
    print(f" LOL:\n{lol}")
    return row, lol

# call the function
_, table = rec(d=sparse_sample_dict)


function called; got:
 d: {'ABC': {'open': {}, 'updated': {'12': {'22334': 5, '22335': 5}, '01': {'22335': 1, '22336': 2, '22337': 2}, '02': {'22337': 8, 'order': 2}, '03': {'order': 5}}}, 'DEF': {'open': {}, 'updated': {'12': {'22338': 5, '22339': 5}, '01': {'22339': 1, '22340': 2, '22341': 2}, '02': {'22341': 8, 'order': 2}, '03': {'order': 5}}}};
 ROW: None;
 LOL: None

added key ABC to ROW:
['ABC']

row continues; recursing...

function called; got:
 d: {'open': {}, 'updated': {'12': {'22334': 5, '22335': 5}, '01': {'22335': 1, '22336': 2, '22337': 2}, '02': {'22337': 8, 'order': 2}, '03': {'order': 5}}};
 ROW: ['ABC'];
 LOL: []

added key open to ROW:
['ABC', 'open']

ROW ends here; added {} to ROW:
['ABC', 'open', {}]

appended finalized row to LOL:
[['ABC', 'open', {}]]

removed latest entry from ROW again:
['ABC', 'open']

current val {} processed.

removed latest level from ROW:
['ABC']

added key updated to ROW:
['ABC', 'updated']

row continues; recursing...

function calle

In [3]:
# Code from above, without all the explanatory print statements but
#  with nicer naming

def recurse(d, table=None, row=None):
    """Recursively flatten sparse dict `d` into list of lists `table`."""
    
    if not row:
        row = [] 
    if not table:
        table = []
    
    for k, v in d.items():
        row.append(k)
        if v and isinstance(v, dict):
            table, row = recurse(d=v, table=table, row=row)
        else:
            row.append(v)
            table.append(row[:])
            row.pop()
        row.pop()
        
    return table, row


In [4]:
import pandas as pd

# retrieve table content as a list of lists, where each (inner) list
#  equals one row in the final table
table_as_list_of_lists, _ = recurse(sparse_sample_dict)

# convert list of lists (i.e., rows) to pandas.DataFrame
df = pd.DataFrame(table_as_list_of_lists)

# proof
df.head()

,0,1,2,3,4
0,ABC,open,{},None,NaN
1,ABC,updated,12,22334,5.0
2,ABC,updated,12,22335,5.0
3,ABC,updated,01,22335,1.0
4,ABC,updated,01,22336,2.0
